<a href="https://colab.research.google.com/github/orgoca/W207FinalProject/blob/main/Project2EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project 2 Baseline
Quora Project



In [1]:
# # Code to read csv file from Google Drive into Colaboratory:
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# # Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [2]:
#Import Libraries
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score
import csv
import random

In [3]:
# #Load test DF
# testLink = 'https://drive.google.com/file/d/1chKVOcXwNzGRe_hRxpGlq18NVaSRM8X9/view?usp=sharing'
# idTest = testLink.split('/')[-2]
# downloaded = drive.CreateFile({'id':idTest}) 
# downloaded.GetContentFile('test.csv')  

In [4]:
test = pd.read_csv('test.csv')

In [5]:
# #Load train DF
# trainLink = 'https://drive.google.com/file/d/1fP-AWiUGLjLkL0c_u67HOUFmHOMO7qm2/view?usp=sharing'
# idTrain = trainLink.split('/')[-2]
# downloaded = drive.CreateFile({'id':idTrain}) 
# downloaded.GetContentFile('train.csv')  

In [6]:
train = pd.read_csv('train.csv')

In [7]:
# #Split Training Data into MiniTrain and MiniTest (Test data has no labels so impossible to measure error from it)
# random.seed(10)
# miniTrain, miniTest = train_test_split(train, test_size=0.2)

# miniTrain.to_csv('miniTrain.csv', index=False)

# miniTest.to_csv('miniTest.csv', index=False)

In [8]:
#Even with a seed the randomization is messing up predictions since miniTest will not find the same questions further down, 
#so rather load a static version from a csv instead.

miniTrain = pd.read_csv('miniTrain.csv')

miniTest = pd.read_csv('miniTest.csv')

In [9]:
#Create a vectorizer, create question and target lists, and vectorize train questions
vectorizer = CountVectorizer()
trainQuestions = miniTrain.question_text.tolist()
trainLabels = miniTrain.target.tolist()
vTrain = vectorizer.fit_transform(trainQuestions)

In [10]:
#What is the vocabulary size, and create a bag of words
print("vocab size: ", vTrain.shape)
bagOfWords = vectorizer.vocabulary_
len(bagOfWords)

vocab size:  (1044897, 173498)


173498

In [11]:
#Create a logistic regression model and fit to train data
lr_clf = LogisticRegression(C=1)
lr_clf.fit(vTrain, trainLabels)

train_predict = lr_clf.predict(vTrain)
train_predict

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array([0, 0, 0, ..., 0, 0, 0])

In [12]:
lr_clf.predict_proba(vTrain)

array([[0.99759451, 0.00240549],
       [0.99804726, 0.00195274],
       [0.99679985, 0.00320015],
       ...,
       [0.99833804, 0.00166196],
       [0.99491892, 0.00508108],
       [0.99879349, 0.00120651]])

In [13]:
#Give this a try with your own questions, it's good fun!
#Try an insincere question. Insincere questions should get a higher probability score to legitimate questions.
testQuestionInsincere1 = "Why are Indians a superior race to Pakistanis?"

vdev = vectorizer.transform([testQuestionInsincere1.lower()])

lr_clf.predict_proba(vdev)[0][1]

0.9434353280690533

In [14]:
#Try a sincere question. Sincere questions should get a lower probability score ideally close to zero.
testQuestionInsincere2 = "What are the origins of the conflict between Indians and Pakistanis?"
vdev = vectorizer.transform([testQuestionInsincere2.lower()])

lr_clf.predict_proba(vdev)[0][1]

0.2063788732333306

In [15]:
# #DON'T run this section. It takes too long. I've produced a CSV from the output and you can run from there by loading such CSV in the next block
# #Let's predict probability for each question in the miniTest dataframe
# columnNames = ['question_text', 'pred']
# predictDF = pd.DataFrame(columns = columnNames)

# for question in miniTest.question_text: 
#   vdev = vectorizer.transform([question.lower()])
#   pred = lr_clf.predict_proba(vdev)[0][1]
#   predictDF = predictDF.append({'question_text':question,'pred': pred}, ignore_index=True)

# predictDF.to_csv('predictDF.csv', index=False)

In [16]:
predictDF = pd.read_csv('predictDF.csv')

predictDF

,question_text,pred
0,Did Jimi Hendrix really even care about or hav...,0.019599
1,How many three-digit numbers are there such th...,0.003087
2,What kind of geographical conditions are requi...,0.001420
3,You find a book with your life summarized insi...,0.006146
4,What are the main reasons behind public hair f...,0.007620
...,...,...
261220,What are some cultural similarities between ol...,0.003478
261221,What happens to the products of respiration?,0.010044
261222,What is the accounting formula for closing stock?,0.000633
261223,Will saying somthing in anger to others comes ...,0.013100


In [17]:
miniTest

,qid,question_text,target
0,511b6fa03ab9f88e249e,Did Jimi Hendrix really even care about or hav...,0
1,1d9400d2d79939a44443,How many three-digit numbers are there such th...,0
2,a48dc7bd82feea5d52ca,What kind of geographical conditions are requi...,0
3,819549ba01220c43c69e,You find a book with your life summarized insi...,0
4,23e725d16a11719023a1,What are the main reasons behind public hair f...,0
...,...,...,...
261220,b1aa7b30fc81ecc62e10,What are some cultural similarities between ol...,0
261221,4805e711e00f0d2e2608,What happens to the products of respiration?,0
261222,3915ca1fdc21fd4a6fe0,What is the accounting formula for closing stock?,0
261223,0321a0723ee8bc9f35e0,Will saying somthing in anger to others comes ...,0


In [18]:
# #Let's see some nasty questions
# nastyQuestions = predictDF[predictDF.pred > .9]

# nastyQuestions.to_csv('nastyQuestions.csv', index=False)

In [19]:
#Merge predict DF to miniTestDF to produce prediction metrics

summaryDF = pd.merge(predictDF,miniTest,on='question_text',how='left')

# summaryDF.to_csv('summaryDF.csv', index=False)

summaryDF

,question_text,pred,qid,target
0,Did Jimi Hendrix really even care about or hav...,0.019599,511b6fa03ab9f88e249e,0
1,How many three-digit numbers are there such th...,0.003087,1d9400d2d79939a44443,0
2,What kind of geographical conditions are requi...,0.001420,a48dc7bd82feea5d52ca,0
3,You find a book with your life summarized insi...,0.006146,819549ba01220c43c69e,0
4,What are the main reasons behind public hair f...,0.007620,23e725d16a11719023a1,0
...,...,...,...,...
261220,What are some cultural similarities between ol...,0.003478,b1aa7b30fc81ecc62e10,0
261221,What happens to the products of respiration?,0.010044,4805e711e00f0d2e2608,0
261222,What is the accounting formula for closing stock?,0.000633,3915ca1fdc21fd4a6fe0,0
261223,Will saying somthing in anger to others comes ...,0.013100,0321a0723ee8bc9f35e0,0


In [20]:
confusionMatrixDF = summaryDF[['pred', 'target']]

#Tried -by hand- values between 0.5:1.0 and 0.2 produced the best F1 score

confusionMatrixDF['pred'] = np.where(confusionMatrixDF['pred'] > 0.2, 1, 0)

confusionMatrixDF

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,pred,target
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
261220,0,0
261221,0,0
261222,0,0
261223,0,0


In [21]:
#Compute baseline F1 Score (and other metrics)
confusionMatrixDF

print('Accuracy:', accuracy_score(confusionMatrixDF.target, confusionMatrixDF.pred))
print('F1 score:', f1_score(confusionMatrixDF.target, confusionMatrixDF.pred))
print('Recall:', recall_score(confusionMatrixDF.target, confusionMatrixDF.pred))
print('Precision:', precision_score(confusionMatrixDF.target, confusionMatrixDF.pred))
print('\n Clasification report:\n', classification_report(confusionMatrixDF.target,confusionMatrixDF.pred))
print('\n Confussion matrix:\n',confusion_matrix(confusionMatrixDF.target, confusionMatrixDF.pred))

Accuracy: 0.9469882285386161
F1 score: 0.6202489990676247
Recall: 0.6999876206981926
Precision: 0.5568193008370261

 Clasification report:
               precision    recall  f1-score   support

           0       0.98      0.96      0.97    245069
           1       0.56      0.70      0.62     16156

    accuracy                           0.95    261225
   macro avg       0.77      0.83      0.80    261225
weighted avg       0.95      0.95      0.95    261225


 Confussion matrix:
 [[236068   9001]
 [  4847  11309]]
